In [1]:
#Author: kannan
import pandas as pd
from textblob import TextBlob
from datetime import datetime

#### reading from consolidated excel file

In [2]:
in_filepath='../data/output/SS_Extracted_content.xlsx'
out_filepath='../data/output/2_Sentiment_Polarity_Subjectivity.xlsx'
df=pd.read_excel(in_filepath)
df.head()

,Id,url,title,html,content
0,1,http://mashable.com/2014/09/01/americans-held-...,Americans Held in North Korea Ask U.S. for Ass...,<!DOCTYPE html>\n<html data-env='production' l...,"PYONGYANG, North Korea — North Korea gave fore..."
1,2,http://mashable.com/2014/09/01/apple-visa-mast...,"Apple Partners With Visa, MasterCard, AmEx for...",<!DOCTYPE html>\n<html data-env='production' l...,According to new reports from Bloomberg and ot...
2,3,http://mashable.com/2014/09/01/aussie-football...,Aussie Football Players Dress Up as Rolf Harri...,<!DOCTYPE html>\n<html data-env='production' l...,SYDNEY — Two Australian football players are a...
3,4,http://mashable.com/2014/09/01/australia-gover...,Australia Government Buys Bombproof BMW Fleet ...,<!DOCTYPE html>\n<html data-env='production' l...,SYDNEY — The Australia Prime Minister has orde...
4,5,http://mashable.com/2014/09/01/australia-jane-...,Author Compares Women in Traditional Marriages...,<!DOCTYPE html>\n<html data-env='production' l...,"SYDNEY — On Australian panel show Q&A, author ..."


In [3]:
df.shape

(7795, 5)

#### calculating subjectivity and 

In [4]:
df.iloc[0]['content']

'PYONGYANG, North Korea — North Korea gave foreign media access on Monday to three detained Americans who said they have been able to contact their families and — watched by officials as they spoke — called for Washington to send a high-ranking representative to negotiate for their freedom.\nJeffrey Fowle and Mathew Miller said they expect to face trial within a month. But they said they do not know what punishment they could face or what the specific charges against them are. Kenneth Bae, who already is serving a 15-year term, said his health has deteriorated at the labor camp where he works eight hours a day.\nSee also: North Korea\'s Marathon: A Rare Chance to Tour Pyongyang on Foot\nThe three were allowed to speak briefly with The Associated Press at a meeting center in Pyongyang. North Korean officials were present during the interviews, conducted separately and in different rooms, but did not censor the questions that were asked. The three said they did not know they were going t

In [5]:
#kannan: TextBlob tutorial: https://textblob.readthedocs.io/en/dev/quickstart.html
tmp=TextBlob(df.iloc[0]['content'])
tmp.sentiment

Sentiment(polarity=0.013541666666666674, subjectivity=0.3909722222222222)

In [6]:
#kannan: If required we trip the content before calculating sentiment (e.g content=left(content, 1000))
def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

def mid(s, offset, amount):
    return s[offset:offset+amount]

In [7]:
#Iterate through all documents
df['polarity_title']=0
df['subjectivity_title']=0
df['polarity_content']=0
df['subjectivity_content']=0

startTimeModule = datetime.now()
for idx, row in df.iterrows():
    #if idx>100: #exit condition
    #    break
    title=TextBlob(str(row['title']))
    content=TextBlob(str(row['content']))
    
    df.loc[idx,'polarity_title']=title.sentiment.polarity
    df.loc[idx,'subjectivity_title']=title.sentiment.subjectivity
    
    df.loc[idx,'polarity_content']=content.sentiment.polarity
    df.loc[idx,'subjectivity_content']=content.sentiment.subjectivity

print ('Sentiment Polarity & Subjectivity extraction time:',(datetime.now() - startTimeModule))

Sentiment Polarity & Subjectivity extraction time: 0:00:33.972046


In [8]:
df.head()

,Id,url,title,html,content,polarity_title,subjectivity_title,polarity_content,subjectivity_content
0,1,http://mashable.com/2014/09/01/americans-held-...,Americans Held in North Korea Ask U.S. for Ass...,<!DOCTYPE html>\n<html data-env='production' l...,"PYONGYANG, North Korea — North Korea gave fore...",0.000,0.00,0.013542,0.390972
1,2,http://mashable.com/2014/09/01/apple-visa-mast...,"Apple Partners With Visa, MasterCard, AmEx for...",<!DOCTYPE html>\n<html data-env='production' l...,According to new reports from Bloomberg and ot...,0.000,0.00,0.056136,0.518788
2,3,http://mashable.com/2014/09/01/aussie-football...,Aussie Football Players Dress Up as Rolf Harri...,<!DOCTYPE html>\n<html data-env='production' l...,SYDNEY — Two Australian football players are a...,-0.075,0.05,0.003831,0.348095
3,4,http://mashable.com/2014/09/01/australia-gover...,Australia Government Buys Bombproof BMW Fleet ...,<!DOCTYPE html>\n<html data-env='production' l...,SYDNEY — The Australia Prime Minister has orde...,0.000,0.00,0.124162,0.378684
4,5,http://mashable.com/2014/09/01/australia-jane-...,Author Compares Women in Traditional Marriages...,<!DOCTYPE html>\n<html data-env='production' l...,"SYDNEY — On Australian panel show Q&A, author ...",0.000,0.75,0.070928,0.486269


In [9]:
#writing the output
#kannan: we can remove url, title, content. just keep id and calculated columns. later we can join using 'id'
df.drop(['url', 'title','html','content'], axis=1,inplace=True)
df.to_excel(out_filepath, index=False)